In [ ]:
def euclidean_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def is_convergence(cendt1, cendt2):
    for c1, c2 in zip(cendt1, cendt2):
        for pv in range(3):
            if c1[pv] < c2[pv] - 5 or c1[pv] > c2[pv] + 5:
                return False
    return True

def get_pix(idx, width):
    row = idx // width
    col = idx % width
    return row, col
    
def kmeans(img_1d, k_clusters, max_iter, init_centroids='random'):
    height, width, _ = img_1d.shape
    
    centroids = None
    if init_centroids == 'random':
        centroids = [[random.randint(0, 256) for _ in range(3)] for _ in range(k_clusters)]
    elif init_centroids == 'in_pixels':
        centroids = [img_1d[random.randint(0, height)][random.randint(0, width)] for _ in range(k_clusters)]
    else:
        raise ValueError('init_centroids must be "random" or "in_pixels"')
    
    assigned = [-1] * (height * width)
    # loop until get results or reach max iteration
    for _ in range(max_iter):
        clusters = [[] for _ in range(k_clusters)]
        # calculate the distances between pixel and centroits - assign to cluster
        dis_centd = [[-1 for _ in range(k_clusters)]] * (height * width)

        for i in range(height * width):
            min_distance = float('inf')
            pos = -1
            row, col = get_pix(i, width)
            for j in range(k_clusters):
                dis_centd[i][j] = euclidean_distance(centroids[j], img_1d[row][col])
                if dis_centd[i][j] < min_distance:
                    min_distance = dis_centd[i][j]
                    pos = j
            assigned[i] = pos
            clusters[pos].append(list(img_1d[row][col]))
        
        # update centroids
        new_centroids = [np.mean(data, axis=0) if data else [0, 0, 0] for data in clusters]        
        for c in new_centroids:
            if all(elem == 0 for elem in c):
                c = [random.randint(0, 256) for _ in range(3)]
        
        # checking convergence, completeness
        if is_convergence(centroids, new_centroids):
            break
        else:
            del centroids
            centroids = new_centroids
    
    # generate labels
    arr = [[-1] * width] * height
    labels = np.array(arr)
    for idx, a in enumerate(assigned):
        row, col = get_pix(idx, width)
        labels[row][col] = a
    
    return centroids, labels

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random

def main():
    # load image
    img_path = str(input('Path to image: '))
    try:
        image = Image.open(img_path)
    except:
        print('can not open image!')
        return
    
    inp_img = np.array(image)
    shape = inp_img.shape
    
    # console UI
    print('\nK-means Clustering algorithm for color compression')    
    k_clusters = int(input('Number of clusters: '))
    max_iters = int(input('Max iterations: '))

    centroids, labels = kmeans(inp_img, k_clusters, max_iters, 'in_pixels')

    seg_img = np.array(image)
    for i in range(shape[0]):
        for j in range(shape[1]):
            seg_img[i][j] = centroids[labels[i][j]].copy()    
    
    # display the original and segmented images
    fig, axs = plt.subplots(2, 1, figsize=(26, 14))
    axs[0].imshow(inp_img)
    axs[0].set_title("Original Image")
    axs[0].axis("off")

    axs[1].imshow(seg_img)
    axs[1].set_title("Segmented Image")
    axs[1].axis("off")

    plt.tight_layout()
    plt.show()
    
    # saving image option
    print('\n\nChose an extension for saving segmented image: ')
    print('1. png')
    print('2. pdf')
    print('0. no saving')
    ext = int(input('Your choice: '))
    
    if ext == 1:
        plt.imsave('output.png', seg_img)
    elif ext == 2:
        plt.imsave('output.pdf', seg_img)
    else:
        return
    
if __name__ == "__main__":
    main()